In [11]:
import pandas as pd
import minsearch

C:\Users\wifqo\Documents\PROJECTS\fitness-assistant\notebooks\minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [2]:
df = pd.read_csv("../data.csv")

In [3]:
df

,id,exercise_name,type_of_activity,type_of_equipment,body_part,type,muscle_groups_activated,instructions
0,0,Push-Ups,Strength,Bodyweight,Upper Body,Push,"Pectorals, Triceps, Deltoids",Start in a high plank position with your hands...
1,1,Squats,Strength,Bodyweight,Lower Body,Push,"Quadriceps, Glutes, Hamstrings",Stand with feet shoulder-width apart. Lower yo...
2,2,Plank,Strength/Mobility,Bodyweight,Core,Hold,"Rectus Abdominis, Transverse Abdominis",Start in a forearm plank position with your el...
3,3,Deadlift,Strength,Barbell,Lower Body,Pull,"Glutes, Hamstrings, Lower Back","Stand with feet hip-width apart, barbell in fr..."
4,4,Bicep Curls,Strength,Dumbbells,Upper Body,Pull,"Biceps, Forearms","Stand with a dumbbell in each hand, arms fully..."
...,...,...,...,...,...,...,...,...
202,202,Incline Dumbbell Row,Strength,Dumbbells,Upper Body,Pull,"Latissimus Dorsi, Biceps",Lie face down on an incline bench with a dumbb...
203,203,Machine Lat Pulldown,Strength,Machine,Upper Body,Pull,"Latissimus Dorsi, Biceps",Sit at a lat pulldown machine with a wide grip...
204,204,One-Arm Cable Row,Strength,Cable Machine,Upper Body,Pull,"Latissimus Dorsi, Biceps",Stand facing a cable machine with the handle a...
205,205,Face Pull,Strength,Cable Machine,Upper Body,Pull,"Rear Deltoids, Trapezius, Rhomboids",Attach a rope handle to a high cable pulley. P...


In [12]:
df.columns

Index(['id', 'exercise_name', 'type_of_activity', 'type_of_equipment',
       'body_part', 'type', 'muscle_groups_activated', 'instructions'],
      dtype='object')

In [13]:
index = minsearch.Index(
    text_fields = ['exercise_name', 'type_of_activity', 'type_of_equipment',
       'body_part', 'type', 'muscle_groups_activated', 'instructions'],
    keyword_fields = []
)

In [15]:
documents = df.to_dict(orient='records')

In [18]:
index.fit(documents)

In [19]:
query = 'give me exercises for hamstrings' 

In [20]:
index.search(query)

[{'id': 118,
  'exercise_name': 'Leg Curl',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Machine',
  'body_part': 'Lower Body',
  'type': 'Pull',
  'muscle_groups_activated': 'Hamstrings',
  'instructions': 'Lie face down on a leg curl machine and curl the pad towards your glutes, then return to the starting position.'},
 {'id': 8,
  'exercise_name': 'Hamstring Stretch',
  'type_of_activity': 'Mobility',
  'type_of_equipment': 'Bodyweight',
  'body_part': 'Lower Body',
  'type': 'Stretch',
  'muscle_groups_activated': 'Hamstrings',
  'instructions': 'Stand upright and place one heel on a bench or step. Lean forward slightly to feel a stretch in the hamstring of the elevated leg.'},
 {'id': 95,
  'exercise_name': 'Machine Leg Curl',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Machine',
  'body_part': 'Lower Body',
  'type': 'Pull',
  'muscle_groups_activated': 'Hamstrings',
  'instructions': 'Sit on a leg curl machine and curl the pad down towards your glutes,

In [23]:
import os
from dotenv import load_dotenv
import openai
from openai import OpenAI

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI()

In [24]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{"role": "user", "content": query}]
)

In [25]:
print(response.choices[0].message.content)

Sure! Here are several effective exercises to strengthen and stretch your hamstrings. Always warm up before performing these exercises, and consider consulting a fitness professional if you're new to working out to ensure proper form and avoid injury.

### Strengthening Exercises:

1. **Deadlifts:**
   - **Conventional Deadlift:** Stand with feet hip-width apart, barbell in front. Bend at your hips and knees to grip the barbell, keeping your back straight. Lift by extending your hips and knees simultaneously.
   - **Romanian Deadlift:** Hold a barbell or dumbbells in front of your thighs. With a slight bend in your knees, hinge at your hips to lower the weights down your legs while keeping your back straight. Return to standing.

2. **Hamstring Curls:**
   - **Machine Hamstring Curl:** Adjust the machine seat and leg pad to fit. Curl your legs up against resistance and slowly return to the starting position.
   - **Stability Ball Hamstring Curl:** Lie on your back with your feet on a s

In [28]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )
    return results

In [29]:
prompt_template = """
        You emulate a user of our fitness assistant application.
    Formulate 5 questions this user might ask based on a provided exercise.
    Make the questions specific to this exercise.
    The record should contain the answer to the questions, and the questions should
    be complete and not too short. Use as fewer words as possible from the record. 

        QUESTION: {question}

        CONTEXT: {context}
""".strip()

entry_template = """
    exercise_name: {exercise_name}
    type_of_activity: {type_of_activity}
    type_of_equipment: {type_of_equipment}
    body_part: {body_part}
    type: {type}
    muscle_groups_activated: {muscle_groups_activated}
    instructions: {instructions}
""".strip()


def build_prompt(query, search_results):
    
    context = ""

    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt
    
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [30]:
rag(query)

'1. How do I perform a Lying Leg Curl exercise on a machine?\n   - Lie face down on a leg curl machine and curl the pad towards your glutes, then return to the starting position.\n\n2. What type of activity is the Prone Leg Curl classified as?\n   - Prone Leg Curl is classified as a Strength activity.\n\n3. Can you tell me which muscle is primarily activated during a Machine Leg Curl?\n   - The primary muscle activated is the hamstrings.\n\n4. Which equipment is needed for the Seated Leg Curl exercise?\n   - You need a machine for the Seated Leg Curl exercise.\n\n5. Is the Leg Curl exercise a push or a pull type of movement?\n   - The Leg Curl is a pull type of movement.'